Este agarra los datos del csv con la matriz de grayscale de cada espectrograma, y se los pasa al modelo de CNN de PyTorch (definido en aux file models.py), los guarda para poder llamarlo sin re-entrenar desde otros archivos

In [ ]:
import os, sys
import pandas as pd
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import torch.nn.functional as F

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# New clean training API
from utils.training_core import cross_val_training, single_fold_training
from utils.models import BirdCNN
from utils.evaluation_utils import plot_kfold_results, plot_single_fold_curve, print_single_fold_results

# Legacy imports for backward compatibility
from utils.dataset_utils import StandardizedDataset
from utils.training_utils import train_single_fold, single_fold_training_with_predefined_split
from utils.cross_validation import k_fold_cross_validation_with_predefined_folds
import utils.split as split

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if device.type == 'cuda':
    print(torch.cuda.get_device_name(device))
else:
    print("CUDA not available")

Using device: cuda
NVIDIA GeForce RTX 5080


In [ ]:
# UNCOMMENT THE ONE BEING USED

# NOT NOISE-REDUCED
df = pd.read_csv(os.path.join('..', 'database', 'meta', 'final', 'train_data.csv'))

# NOISE-REDUCED
# df = pd.read_csv(os.path.join('..', 'database', 'meta', 'final', 'train_data2.csv'))

In [ ]:
# Extract labels, authors, and pixel values
labels = df['label'].values.astype(np.int64)
authors = df['author'].values
features = df.drop(columns=['label', 'author']).values.astype(np.float32)
# Convert to 0-1 range first, then standardization will be applied per fold
features /= 255.0
features = features.reshape(-1, 1, 313, 224)

print("features shape:", features.shape)
print("labels shape:", labels.shape)
print("authors shape:", authors.shape)

# Create metadata DataFrame for splitting (with sample indices)
metadata_df = pd.DataFrame({
    'sample_idx': range(len(df)),
    'class_id': labels,
    'author': authors,
    'usable_segments': 1  # Each sample represents 1 segment
})

print("metadata_df shape:", metadata_df.shape)
print("Unique authors:", len(metadata_df['author'].unique()))
print("Unique classes:", len(metadata_df['class_id'].unique()))

In [ ]:
plt.figure(figsize=(10, 5))
df['label'].value_counts().sort_index().plot(kind='bar')
plt.xlabel('Label')
plt.ylabel('Number of Samples')
plt.title('Number of Samples per Label')
plt.show()

In [ ]:
import importlib
import utils.models
import utils.dataset_utils
import utils.training_utils
import utils.cross_validation
import utils.evaluation_utils
import utils.split

importlib.reload(utils.models)
importlib.reload(utils.dataset_utils)
importlib.reload(utils.training_utils)
importlib.reload(utils.cross_validation)
importlib.reload(utils.evaluation_utils)
importlib.reload(utils.split)

# Import specific functions
from utils.dataset_utils import StandardizedDataset

In [ ]:
# Data is now handled automatically by the new training API
# The training functions will create the dataset internally
print("Using new clean training API - data preparation is handled automatically")

## K Fold Training with Predefined Splits

In [ ]:
# Split finding is now handled automatically by the new training API
# The cross_val_training function will find optimal splits internally
print("Split finding is now handled automatically by the cross_val_training function")

In [ ]:
print("Fold conversion is now handled automatically by the cross_val_training function")

In [ ]:
# Run K-Fold training using the new clean API
results, best_results = cross_val_training(
    features=features,
    labels=labels,
    authors=authors,
    model_class=BirdCNN,
    num_classes=31,
    config={
        'k_folds': 4,
        'num_epochs': 220,
        'batch_size': 24,
        'learning_rate': 0.001,
        'use_class_weights': False,
        'early_stopping': 35,
        'standardize': True,
        'aggregate_predictions': True,
        'max_split_attempts': 30000,
        'min_val_segments': 0
    }
)

In [ ]:
# Plot accuracy and loss curves
plot_kfold_results(results, best_results)

## Single Fold Training with Predefined Splits

In [ ]:
# Split finding is now handled automatically by the single_fold_training function
print("Split finding is now handled automatically by the single_fold_training function")

In [ ]:
# Run single fold training using the new clean API with predefined split
single_results = single_fold_training(
    features=features,
    labels=labels,
    authors=authors,
    model_class=BirdCNN,
    num_classes=31,
    config={
        'num_epochs': 250,
        'batch_size': 48,
        'learning_rate': 0.001,
        'use_class_weights': False,
        'early_stopping': 35,
        'standardize': True,
        'test_size': 0.2,
        'max_split_attempts': 10000,
        'min_test_segments': 5
    },
    use_predefined_split=True
)

In [ ]:
# Plot individual training curves for single fold
plot_single_fold_curve(single_results, metric_key='accuracies', title="Single Fold - Accuracy Curves", ylabel="Accuracy")
plot_single_fold_curve(single_results, metric_key='losses', title="Single Fold - Loss Curves", ylabel="Cross Entropy Loss")
plot_single_fold_curve(single_results, metric_key='f1s', title="Single Fold - F1 Score Curves", ylabel="Macro F1 Score")

# Print results summary
print_single_fold_results(single_results)

## Single Fold Training with Regular 80-20 Split

In [ ]:
# Run single fold training using the new clean API with regular 80-20 split
single_results_80_20 = single_fold_training(
    features=features,
    labels=labels,
    authors=authors,
    model_class=BirdCNN,
    num_classes=31,
    config={
        'num_epochs': 250,
        'batch_size': 48,
        'learning_rate': 0.001,
        'use_class_weights': False,
        'early_stopping': 35,
        'test_size': 0.2,
        'random_state': 435
    },
    use_predefined_split=False
)

In [ ]:
# Plot individual training curves for 80-20 split
plot_single_fold_curve(single_results_80_20, metric_key='accuracies', title="80-20 Split - Accuracy Curves", ylabel="Accuracy")
plot_single_fold_curve(single_results_80_20, metric_key='losses', title="80-20 Split - Loss Curves", ylabel="Cross Entropy Loss")
plot_single_fold_curve(single_results_80_20, metric_key='f1s', title="80-20 Split - F1 Score Curves", ylabel="Macro F1 Score")

# Print results summary
print_single_fold_results(single_results_80_20)

leave this code commented for now

In [ ]:
# import cProfile

# cProfile.run(
#     "util.k_fold_cross_validation(dataset=dataset, model_class=models.BirdCNN, num_classes=28, k_folds=5, num_epochs=250, batch_size=48, lr=0.001, aggregate_predictions=True, random_state=435, use_class_weights=True)",
#     filename="../profiler/profile_output.prof"
# )

In [ ]:
# # Run K-Fold training with standardization
# results_unb, best_runb = util.k_fold_cross_validation(
#     dataset=dataset,
#     model_class=models.BirdCNN,
#     num_classes=28,
#     k_folds=5,
#     num_epochs=220,
#     batch_size=24,
#     lr=0.001,
#     aggregate_predictions=True,
#     random_state=1789,
#     use_class_weights=False,
#     standardize=True
# )

In [ ]:
# util.plot_kfold_results(results_unb, best_runb)

In [ ]:
# # Select Name, Best Fold, and Model
# model_name = 'bird_cnn'
# best_model_state = results['fold_results']['fold_3']['model_state']
# model = models.BirdCNN(num_classes=28).to(device)

# # ============= Don't modify below this line ============= Ensure variables above are set correctly ============
# model.load_state_dict(best_model_state)
# model_dir = os.path.join('..', 'models')
# os.makedirs(model_dir, exist_ok=True)
# save_path = os.path.join(model_dir, f"{model_name}.pth")

# util.save_model(model, model_name, model_save_path=save_path)
# util.test_saved_model(save_path)

Call Single Model

In [ ]:
# # Run single fold 80-20 training
# single_results = util.single_fold_training(
#     dataset=dataset,
#     model_class=models.BirdCNN,
#     num_classes=28,
#     num_epochs=250,
#     batch_size=48,
#     lr=0.001,
#     test_size=0.2,
#     random_state=435,
#     use_class_weights=True
# )

In [ ]:
# # Plot individual training curves
# util.plot_single_fold_curve(single_results, metric_key='accuracies', title="Single Fold - Accuracy Curves", ylabel="Accuracy")
# util.plot_single_fold_curve(single_results, metric_key='losses', title="Single Fold - Loss Curves", ylabel="Cross Entropy Loss")
# util.plot_single_fold_curve(single_results, metric_key='f1s', title="Single Fold - F1 Score Curves", ylabel="Macro F1 Score")

# # Print results summary
# util.print_single_fold_results(single_results)

# # Save the trained model
# util.save_model(single_results['model'], "bird_cnn_single_fold", model_save_path=os.path.join(model_dir, "bird_cnn_single_fold.pth"))